In [1]:
# Standard library imports
import os
from datetime import datetime

# Third-party library imports
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from skimage.metrics import structural_similarity as ssim
from torch.distributions import MultivariateNormal
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image
from tqdm import tqdm

# Local application/library specific imports

from objectives import compute_elbo_dist as compute_elbo#, compute_elbo_mmd,compute_elbo_dreg
from test_functions import calculate_fid_routine
from utils import Constants, initialize_weights, unpack_data_biked


True


In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
device = 'cuda'

In [4]:
use_resnet = False
if use_resnet:
    from models.gmrf_mvae_biked import Biked_GMMVAE
else:
    from models.gmrf_mvae_biked_simpler import Biked_GMMVAE

In [5]:
resolution = 64

num_workers = 4
hidden_dim=128
n_layers=2
latent_dim=4
diagonal_transf = 'softplus'#'exp'#relu
reduced = False
centered = False

In [6]:
fid_path = f'../fid_{resolution}'
inception_module_path = '/NAS/USERS/e020199/BOMRGD/phd_fouad/mrf_mvae_project/data/pt_inception-2015-12-05-6726825d.pth'
datadir = f'../biked_data/BIKED Dataset/BIKED Dataset/Segmented bike images_res{resolution}x{resolution}'

In [7]:
class Params():
    def __init__(self,latent_dim=latent_dim, diagonal_transf =diagonal_transf):
        self.latent_dim = latent_dim
        self.diagonal_transf = diagonal_transf
        self.tmpdir = datadir
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.device = device
        self.reduced_diag = reduced
        self.centered = centered
        self.inception_module_path = inception_module_path
        # self.n_layers_vae = n_layers_vae
params = Params()

In [13]:
model = Biked_GMMVAE(params).to(device)
# load_path = '../runs/gmrf/resnet/ep_100_beta_0.001_lt_4_n_2_hd_128_tf_softplus_20250121_225749/model_state_dict_99.pt'
load_path = '../runs/gmrf/simpler_resnet/ep_100_beta_0.001_lt_4_n_2_hd_128_tf_softplus_20250122_195712/model_state_dict_99.pt'
model.load_state_dict(torch.load(load_path),strict=False)

iside latest model
off_diag_scale  1


<All keys matched successfully>

In [14]:
count_parameters(model)

6256775

In [15]:
16*5*2+20, 4*2*2*5

(180, 80)

In [16]:
print(error)

NameError: name 'error' is not defined

In [17]:
batch_size = 64
train_loader, test_loader = model.getDataLoaders(batch_size = batch_size, num_workers = num_workers, pin_memory = False)

In [18]:
# dataT = next(iter(test_loader))
# data = unpack_data_biked(dataT)

In [19]:
# for i in range(5):
#     for j in range(5):
#         model.conditional_generate(data[i],j,i,1)

In [20]:
mean_fid_randm, mean_fid_condgen = calculate_fid_routine(model, test_loader, params, datadir, fid_path, num_fid_samples= 5000, device=device)

[INFO] Setting up FID directories...
[INFO] Generating unconditional (random) samples...


Uncond: 100%|██████████| 50/50 [02:16<00:00,  2.72s/it]

[INFO] Generating conditional samples...



Cond:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([3, 1, 64, 64])
torch.Size([3, 1, 64, 64])
torch.Size([3, 1, 64, 64])
torch.Size([3, 1, 64, 64])
torch.Size([3, 1, 64, 64])


Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init(

[INFO] Computing Inception features for real & generated images...
[INFO] Computing real activations for modality='crank' with 0 images...


ZeroDivisionError: integer division or modulo by zero

In [ ]:
%debug

In [ ]:
mean_fid_randm, mean_fid_condgen